In [1]:
#Creates a class for chalets that takes in serial number of chalet, its cost and its capacity
class Chalets:
    def __init__(self):
        self.cost = 0
        self.capacity = 0
        self.serial = 0
        self.assigned_students = []    #This is the list of all students that are assigned to a particular chalet
        self.cant_be_assigned_students = []*100  #This is the list of all students that can't be assigned   
        

    def addChalet(self, serial, cost, capacity):
        self.cost = cost
        self.capacity = capacity
        self.serial = serial
        self.assigned_students = []*capacity    #This is the list of all students that are assigned to a particular chalet
        self.cant_be_assigned_students = []*100  #This is the list of all students that can't be assigned   
        
    def addStudent(self, student):
        if student.serial_number in self.cant_be_assigned_students: #This function checks whether a student is in existing list of cant be assigned students
            pass
        else:
            if len(self.assigned_students) < self.capacity and student.isAssigned == False: #Checks whether number of already assigned students are less than capacity 
                if len(set(student.conflicts).intersection(set(self.assigned_students))) == 0 :
                    self.assigned_students.append(student.serial_number)
                    self.cant_be_assigned_students.extend(student.conflicts)
                    student.isAssignedToChalet(True)
            else:
                pass

    def removeStudent(self, student):
        if student.serial_number in self.assigned_students:
            self.assigned_students.remove(student.serial_number)
            self.capacity -= 1
            for i in range(len(student.conflicts)):
                self.cant_be_assigned_students.remove(student.conflicts[i])
            student.isAssigned = False
                
    def __str__(self):
        return "Chalet with {} serial number has {} students in it. They are {}".format(self.serial, len(self.assigned_students), self.assigned_students)


class Student:
    def __init__(self):
        self.serial_number = 0
        self.conflicts = []  #List of all students' serial number having conflict with given student
        self.number_conflicts = 0
        self.isAssigned = False

    def addStudent(self, serial_number, number_conflicts, list_of_conflicts):
        self.serial_number = serial_number
        self.conflicts = []*number_conflicts  #List of all students' serial number having conflict with given student
        self.number_conflicts = number_conflicts
        self.conflicts.extend(list_of_conflicts)

    def isAssignedToChalet(self, Boolean):
        self.isAssigned = Boolean #True if assigned else false 

    def __str__(self):
        return "Student has {} serial number. She is in conflict with {} students. Their serial numbers are {}".format(self.serial_number, self.number_conflicts, self.conflicts)
        




    

In [2]:
####Genetic Algorithm############

import random

###Reading Files and Creating Chalet and Student List:
  #Reads a given file and returns Chalets Lists and Students Lists. Finds number of chalets and number of students 

def readFile(filename):
    f  = open(filename, 'r')
    M, N = map(int, f.readline().split())
        
    #Creates list of chalets object. Each object is of class chalet 
    Chalets_list = []
    for i in range(1, M+1):
        cost, capacity = map(int, f.readline().split())
        chalet = Chalets()
        chalet.addChalet(i, cost, capacity)
        Chalets_list.append(chalet)
        
    #Creates list of student objects. Each object is of class student 
    Students_list = []
    for i in range(0 , N):
        a = f.readline().split()
        number_conflicts = int(a[0])
        list_of_conflicts = []*number_conflicts
        for j in range(1, number_conflicts+1):
            list_of_conflicts.append(int(a[j]))
        student = Student()
        student.addStudent(i, number_conflicts, list_of_conflicts)
        Students_list.append(student)

    #Closing the opened file 
    f.close()
    
    return Chalets_list, Students_list, M

#Initializing population  
def createChromosome(filename):

    #new list is being created everytime function is called 
    Chalets_list, Students_list, M = readFile(filename)
    
        #introducing randomization
    random.shuffle(Students_list)

        #Renaming original list
    new_student_list = Students_list
        
        #Shuffling Chalet's list
    random.shuffle(Chalets_list)
        
        #Saving Chalet's list as new list
    new_chalets_list = Chalets_list
          
    for student in new_student_list:
        if not student.isAssigned:  # Check if the student is not already assigned   
            
            # First priority: Choose from rented chalets with highest available places
            #Finding chalet with maximum available space and is rented 
            rented_chalet_with_high_place = Chalets()
                
            for chalet in new_chalets_list:
                if len(chalet.assigned_students) > 0: #ensures that chalet is rented

                        #This ensures that chalet having available space most is selected  
                    if (chalet.capacity - len(chalet.assigned_students)) > (rented_chalet_with_high_place.capacity - len(rented_chalet_with_high_place.assigned_students)):
                        rented_chalet_with_high_place = chalet
                
            rented_chalet_with_high_place.addStudent(student)
            # Exit the loop once assigned to a chalet
                    
            # Second priority: Choose from unrented chalets based on capacity/cost score
        if not student.isAssigned:
            max_score = 0
            selected_chalet = None
                
            for chalet in new_chalets_list:
                if len(chalet.assigned_students) == 0:  # Check if chalet is unrented
                        
                    score = chalet.capacity / chalet.cost
                        
                    if score > max_score:
                        max_score = score
                        selected_chalet = chalet
                        
            if selected_chalet is not None:
                selected_chalet.addStudent(student)
                student.isAssigned = True
                
        # Attempt to optimize by exchanging students between cottages       
    for j in range(0,len(new_chalets_list)):
            
        if len(new_chalets_list[j].assigned_students) > 0:  # Only consider rented cottages
                
            num_students_in_j = len(new_chalets_list[j].assigned_students)
                
            for k in range(0, len(new_chalets_list)):
                if len(new_chalets_list[k].assigned_students) == 0:  # Only consider non-rented cottages

                        #checking if unrented cottage capacity is more than students to be relocated and its cost is lesser 
                    if new_chalets_list[k].capacity >= num_students_in_j and new_chalets_list[k].cost < new_chalets_list[j].cost:
                            
                            
                    # Reallocate students from j to cheapest_k
                        for student_serial in new_chalets_list[j].assigned_students[:]:
                            for student in new_student_list: 
                                if student.serial_number == student_serial:
                                    new_chalets_list[j].removeStudent(student)        
                                    new_chalets_list[k].addStudent(student)

        
    return new_chalets_list, new_student_list

#creates population 
def generatePopulation(filename, pop_size):

    #pop is 2D list 
    pop = [[] for _ in range(pop_size)]
    n = 0
    while n < pop_size:

        #A population is created for every list item 
        new_chalets_list3, new_student_list3 = createChromosome(filename)
        
        #Creating chromosomes 
        new_student_list3.sort(key = lambda student : student.serial_number)
        
        for student in new_student_list3:
            for chalet in new_chalets_list3:
                if student.serial_number in chalet.assigned_students:
                    pop[n].append(chalet.serial)
        
        n += 1
    return pop

In [3]:
#Evaluating fitness of chromosomes 

def fitness_function(population, filename):

    #Generating new Chalet list and Student list from filename
    Chalets_list, Students_list, M = readFile(filename)

    #List of cost of each chromosome for entire population
    chromosome_cost = []
    
    #for every row cost of all occupied chalet is added 
    for row in population:
        cost = 0
        for serial in set(row):
            for chalet in Chalets_list:
                if chalet.serial == serial:
                   cost += chalet.cost

        #Finally it is appended to chromosome cost list 
        chromosome_cost.append(cost)
    
    return chromosome_cost


In [4]:
#Selecting four chromosomes with least cost

def selection(population1, filename, selection_size):

    #Obtain chalet list from readFile:
    Chalets_list, Students_list, M = readFile(filename)
    
    #Obtain chromosome cost from fitness function
    chromosome_cost = fitness_function(population1, filename)

    #Sort it in ascending order
    chromosome_cost.sort()

    #create a 2D list for parents 
    parents = [[] for _ in range(selection_size)]
    
    i = 0

    #Compute cost of each row in population 
    for row in population1:
        cost = 0
        for serial in set(row):
            for chalet in Chalets_list:
                if chalet.serial == serial:
                   cost += chalet.cost

        #if the cost lies within minimum chromosome costs, put those chromosomes in parents 
        if cost in chromosome_cost[0:selection_size]:
            if i < selection_size:
                parents[i].extend(row)
                i += 1
                
    return parents

In [5]:
##Crossing of two individuals 
def reproduction(parents1, crossover_probability):

    #Finding how many genes of parents will be retained in child
    num = int(crossover_probability*len(parents1[0]))

    #Check if number of parents are even else make them even
    if len(parents1)%2 != 0:
        size = len(parents1) - 1
    else:
        size = len(parents1)
        
    #Creating list of children 
    children = [[] for _ in range(size)]
    
    i = 0

    #Creating children which is a mix of parents 
    while i < size:
        children[i] = parents1[i][0:num] + parents1[i+1][num:]
        children[i+1] = parents1[i+1][0:num] + parents1[i][num:]
        i+=2
        
    return children 

In [6]:
##Adding children in original population

def addChildrentoPopulation(population2, parents2, children2):
    i = 0
    for row in population2:

        #checks whether a given row in population is same as parents. 
        if i < len(parents2) and row == parents2[i] :

            #If yes, remove that row and put children row inplace 
            population2.append(children2[i])
            population2.remove(row)
            
            i += 1

    return population2 

In [7]:
#Mutation
def mutation(population3, M, num_genes_mut):
    number = 0
    for row in population3:

        i = 0
        
        while i < num_genes_mut:
            #Generating random integer between 0 and N-1 . This will be student's serial number whose chalet will be changed
            number = random.randint(0, len(row)-1)
    
            #Changing randomly chosen student's assgined chalet to a random chalet 
            row[number] = random.randint(1, M)

            i+=1
            
    return population3

In [8]:
##Repairing####
##Function to call reparing
def checkRepair(mutated_chromosome,filename):

    #Chalet and Student list created using filename 
    new_chalets_list2, new_student_list2 = createChromosome(filename)

    j = 0
    
    for j in range(len(mutated_chromosome)):
        
        for student in new_student_list2:
            for chalet in new_chalets_list2:

                #Checking chalet number assigned to each student and trying to assign it again 
                if chalet.serial == mutated_chromosome[j]:
                    if student.serial_number == j:
                        chalet.addStudent(student)
                        
                        
    #Checking if any student left unassigned
    for student in new_student_list2:
        if student.isAssigned == False:
            return False  
 

    

In [9]:
##Code to return only those solutions which are suitable
def repairedPopulation(population4, filename):

    new_population = []
    
    #Every row in the population acts as a chromosome that has been mutated 
    for i in range(len(population4)):
    
        
        #It will be checked if each row is feasible or not
        if checkRepair(population4[i], filename) != False:
        
            
            #If not feasible that row will be removed          
            new_population.append(population4[i])
            
    #Rest population is sent 
    return new_population

#Find minimum cost
def returnBestSolution(population5, filename):

    #Finds minimum cost out of optimal population
    chromosome_cost5 = fitness_function(population5, filename)
    chromosome_cost5.sort()

    #Obtain chalet list from readFile:
    Chalets_list7, Students_list7, M_new = readFile(filename)

     #Compute cost of each row in population 
    for row in population5:
        cost = 0
        for serial in set(row):
            for chalet in Chalets_list7:
                if chalet.serial == serial:
                   cost += chalet.cost

        #if the cost is minimum chromosome costs, print chromosome and cost 
        if cost == chromosome_cost5[0]:
            return row, chromosome_cost5[0]

In [10]:
#Specify the filename
filename_final = 'instance16_16.txt'

#Creating list of objects for chalets and students 
Chalets_list_final, Students_list_final, M_final  = readFile(filename_final)

#Creating population for given population size , POPULATION SIZE to be specified  
pop0_final = generatePopulation(filename_final, 10000)

#Gives cost of each chromosome
chromosome_cost_final = fitness_function(pop0_final, filename_final)

#Selects parents for matting :SELECTION SIZE to be specified 
parents_final = selection(pop0_final, filename_final, 4000)

#Create children based on above parents. CROSSOVER PROBABILITY to be specified
children_final = reproduction(parents_final, 0.9)

#Updating populating with children 
new_pop_final = addChildrentoPopulation(pop0_final, parents_final, children_final)

#Mutation
mut_pop_final = mutation(new_pop_final, M_final,1 )

#Repaired population
rep_pop_final = repairedPopulation(mut_pop_final, filename_final)

#Solution
chromosome6, cost6 = returnBestSolution(rep_pop_final, filename_final)

print(chromosome6)
print(cost6)

[14, 10, 10, 11, 6, 11, 10, 14, 11, 6, 10, 14, 14, 11, 6, 6]
109
